In [0]:
!pip install confluent_kafka

In [0]:
%restart_python
# OR dbutils.library.restartPython()

In [0]:
import json
import random
import time
from confluent_kafka import Producer

kafka_username = dbutils.secrets.get(scope="kafka-secrets", key="kafka-username")
kafka_password = dbutils.secrets.get(scope="kafka-secrets", key="kafka-password")

# Kafka configuration
conf = {
    'bootstrap.servers': 'pkc-ldvj1.ap-southeast-2.aws.confluent.cloud:9092',
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': kafka_username,
    'sasl.password': kafka_password
}

# Create Producer instance
producer = Producer(conf)

# Function to generate mock market data
def generate_market_data():
    return {
        'symbol': random.choice(['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA']),
        'price': round(random.uniform(100, 1500), 2),
        'volume': random.randint(1, 1000),
        'timestamp': int(time.time())
    }

# Function to send data to Kafka
def send_to_kafka(topic, data):
    producer.produce(topic, key=data['symbol'], value=json.dumps(data))
    producer.flush()

# Generate and send mock data
topic = 'market_data'
for _ in range(100):
    data = generate_market_data()
    send_to_kafka(topic, data)
    time.sleep(1)

In [0]:
# kafka_bootstrap_servers_tls = "pkc-ldvj1.ap-southeast-2.aws.confluent.cloud:9092"
# kafka_username = dbutils.secrets.get(scope="kafka-secrets", key="kafka-username")
# kafka_password = dbutils.secrets.get(scope="kafka-secrets", key="kafka-password")
# topic = "market_data"
# # Read a small batch of data from the Kafka topic
# sample_df = (
#     spark.read
#         .format("kafka")
#         .option("subscribe", topic)
#         .option("kafka.bootstrap.servers", kafka_bootstrap_servers_tls)
#         .option("kafka.security.protocol", "SASL_SSL")
#         .option("kafka.sasl.mechanism", "PLAIN")
#         .option("kafka.sasl.jaas.config", f'kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username="{kafka_username}" password="{kafka_password}";')
#         .option("startingOffsets", "earliest")
#         .option("endingOffsets", "latest")
#         .load()
# )

# # Select and cast the key and value columns to string
# sample_df = sample_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

# # Display the sample data
# display(sample_df)

In [0]:
# # Infer the schema from the sample data
# from pyspark.sql.types import StructType, StructField, StringType

# # Assuming the value column contains JSON data
# json_sample_df = sample_df.select("value")
# json_schema = spark.read.json(json_sample_df.rdd.map(lambda row: row.value)).schema

# # Display the inferred schema
# print(json_schema)

In [0]:
# from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, LongType

# behavioral_input_schema = StructType([
#     StructField("symbol", StringType(), True),
#     StructField("price", DoubleType(), True),
#     StructField("volume", IntegerType(), True),
#     StructField("timestamp", LongType(), True)
# ])